In [ ]:
class Issue:
    def __init__(self, title, description, url, featured_answer):
        self.title = title
        self.description = description
        self.url = url
        self.featured_answer = featured_answer

issues_batch = []

In [3]:
# get repos
# iterate through repos' issues
# iterata through issues' comments and get featured comment by properties : ["author_association"]
# create list of issues objects
# save to parquet
# vecotrize issues and save to vector database

import requests

baseurl = "https://api.github.com/"
token = ""
api_version = "2022-11-28"
headers = {
    "Accept": "application/vnd.github.v3+json",
    "Authorization": f"Bearer {token}",
    "X-GitHub-Api-Version": api_version
}

api = "repositories"
public_repos = requests.get(f"{baseurl}{api}", headers=headers).json()
for repo in public_repos:
    repo_issues = requests.get(f"{baseurl}repos/{repo['full_name']}/issues", headers=headers).json()
    for issue in repo_issues:
        issue_comments = requests.get(f"{baseurl}repos/{repo['full_name']}/issues/{issue['number']}/comments?sort=created&direction=asc", headers=headers).json()
        featured_comment = None
        # should we just get one answer or contact multiple
        for comment in issue_comments:
            if (comment["author_association"] == "OWNER"):
                featured_comment = comment
                break
            elif (comment["author_association"] == "COLLABORATOR"):
                featured_comment = comment
                break
            elif (comment["author_association"] == "MEMBER"):
                featured_comment = comment
                break
            elif (comment["author_association"] == "CONTRIBUTOR"):
                featured_comment = comment
                break
        if featured_comment:
            issues_batch.append(Issue(issue["title"], issue["body"], issue["url"], featured_comment["body"]))


{'message': 'Bad credentials', 'documentation_url': 'https://docs.github.com/rest'}


In [ ]:
# search for poststs in stackoverflow
# get featured answer by properties : ["is_accepted", "score"]
# create list of issues objects
# save all to parquet
# vecotrize posts with is_answered: true and save to vector database